In [ ]:
from df_cereal.df_widget import DFWidget, BaseWidget
import pandas as pd
import pyarrow as pa
import numpy as np
import itertools
import pyarrow.feather as feather
import base64
import io
import time
from ipywidgets import HTML
import threading

In [ ]:
from traitlets import Unicode, List, Dict, observe, Any, Bytes, observe, Bool

class Base64Widget(BaseWidget):
    _view_name = Unicode('Base64WidgetView').tag(sync=True)
    df_base64 = Any("").tag(sync=True)

class BytesWidget(BaseWidget):
    _view_name = Unicode('BytesWidgetView').tag(sync=True)
    df_arrow_bytes = Bytes().tag(sync=True)

class BytesBenchmarkWidget(BaseWidget):
    _view_name = Unicode('BytesBenchmarkWidgetView').tag(sync=True)
    df_arrow_bytes = Bytes().tag(sync=True)
    timing_info = Dict({}).tag(sync=True)
    do_calc = Bool(True).tag(sync=True)

In [ ]:
def df_to_arrow_bytes(df):
    table = pa.Table.from_pandas(df)
    fake_file = io.BytesIO()
    feather.write_feather(table, fake_file, compression='uncompressed')
    fake_file.seek(0)
    return fake_file.read()

def gen_func(exp):
    N = 4 * (10**exp)
    t1 = pd.Timestamp.utcnow()
    big_df = df = pd.DataFrame({
        'one': [1]*N,
        'str': ["foo", "barr", "bazz", "bofffff"] * (N//4),
        'log_normal': np.random.lognormal(25, .3, N)})
    serial_t1 = pd.Timestamp.utcnow()
    byts = df_to_arrow_bytes(big_df)
    serial_t2 = pd.Timestamp.utcnow()
    timing_info = dict(
        serial_t1=serial_t1, serial_t2=serial_t2, gen_time= serial_t1-t1, py_serialize_time=serial_t2-serial_t1,
        rows=N, bytes=len(byts))
    return byts, timing_info

In [ ]:
from buckaroo import BuckarooWidget

In [ ]:
bw = BuckarooWidget(pd.DataFrame())
bw

In [ ]:
bw.raw_df = simple_df

In [ ]:
bw = BuckarooWidget(pd.DataFrame())
display(bw)
def format_results(res):
    tdf = pd.DataFrame(res)
    tdf['py_serialize'] = tdf['serial_t2'] - tdf['serial_t1']
    tdf['widget_xfer'] = tdf['t1'] - tdf['serial_t3']
    tdf['js_deserialize'] = tdf['t2'] - tdf['t1']
    tdf['js_iterate'] = tdf['t3'] - tdf['t2']
    tdf['py_notify'] = tdf['t4'] - tdf['t3']
    tdf['roundtrip'] = tdf['t4'] - tdf['serial_t3']
    bw.raw_df = tdf[['rows', 'bytes', 'widget_xfer', 'js_deserialize', 'js_iterate', 'roundtrip']]
    #display(tdf[['rows', 'bytes', 'widget_xfer', 'js_deserialize', 'js_iterate', 'roundtrip']])
class CallbackBytesBenchmarkWidget(BytesBenchmarkWidget):
    def __init__(self, gen_func, results_arr, min_n, max_n, **kwargs):
        self.results_arr = results_arr
        self.max_n = max_n
        self.current_n = min_n
        byts, self.current_timing = gen_func(self.current_n)
        self.current_timing['serial_t3'] = pd.Timestamp.utcnow()
        super().__init__(df_arrow_bytes=byts)
        self.current_n += 1
        
        
    @observe('timing_info')
    def _timing_info_change(self, change):
        if len(self.timing_info) == 0:
            print("exit early because empty timing_info")
            return
        new_res = { k: pd.Timestamp(v) for k,v in self.timing_info.items()}
        new_res['t4'] = pd.Timestamp.utcnow()
        new_res.update(self.current_timing)
        self.results_arr.append(new_res)
        if self.current_n > self.max_n:
            print("done")
            format_results(results)
            return
        else:
            self.do_calc = False
            self.timing_info = {}
            time.sleep(.1)
            byts, self.current_timing = gen_func(self.current_n)
            print("setting arrow bytes", len(byts) / 1_000_000)

            self.current_timing['serial_t3'] = pd.Timestamp.utcnow()
            self.do_calc = True
            self.df_arrow_bytes = byts
            print("after set arrow bytes")
            self.current_n += 1

results = []
sbw = CallbackBytesBenchmarkWidget(gen_func, results, min_n=1, max_n=7)
sbw

In [ ]:
import buckaroo

In [ ]:
def format_results(res):
    tdf = pd.DataFrame(results)
    tdf['py_serialize'] = tdf['serial_t2'] - tdf['serial_t1']
    tdf['widget_xfer'] = tdf['t1'] - tdf['serial_t3']
    tdf['js_deserialize'] = tdf['t2'] - tdf['t1']
    tdf['js_iterate'] = tdf['t3'] - tdf['t2']
    tdf['py_notify'] = tdf['t4'] - tdf['t3']
    tdf['roundtrip'] = tdf['t4'] - tdf['serial_t3']
    display(tdf[['rows', 'bytes', 'widget_xfer', 'js_deserialize', 'js_iterate', 'roundtrip']])

In [ ]:
while len(results) < 7:
    time.sleep(.5)
tdf = pd.DataFrame(results)
tdf['py_serialize'] = tdf['serial_t2'] - tdf['serial_t1']
tdf['widget_xfer'] = tdf['t1'] - tdf['serial_t3']
tdf['js_deserialize'] = tdf['t2'] - tdf['t1']
tdf['js_iterate'] = tdf['t3'] - tdf['t2']
tdf['py_notify'] = tdf['t4'] - tdf['t3']
tdf['roundtrip'] = tdf['t4'] - tdf['serial_t3']
sbw.close()
tdf[['rows', 'bytes', 'widget_xfer', 'js_deserialize', 'js_iterate', 'roundtrip']]

In [ ]:
4_000_000

In [ ]:
tdf[['rows', 'bytes', 'serial_t1', 'serial_t2',  'widget_xfer', 't1', 'js_deserialize', 't2', 'js_iterate', 't3']]


In [ ]:

def benchmark(df):
    output_area = HTML(
        value="Hello <b>World</b>",
        placeholder='Some HTML2',
        description='Some HTML3')
    display(output_area)
    t1 = pd.Timestamp.utcnow()
    table = pa.Table.from_pandas(df)
    fake_file = io.BytesIO()
    feather.write_feather(table, fake_file, compression='uncompressed')
    fake_file.seek(0)
    full_bytes = fake_file.read()
    t2 = pd.Timestamp.utcnow()
    bbw = BytesBenchmarkWidget(output_area, df_arrow_bytes=full_bytes, serial_t1=t1, serial_t2=t2)
    return bbw

In [ ]:
class CallbackBytesBenchmarkWidget(BytesBenchmarkWidget):
    def __init__(self, gen_func, results_arr, min_n, max_n, **kwargs):
        self.results_arr = results_arr
        self.max_n = max_n
        self.current_n = min_n
        super().__init__(df_arrow_bytes=gen_func(self.current_n))
        self.current_n += 1
        
        
    @observe('timing_info')
    def _timing_info_change(self, change):
        if len(self.timing_info) == 0:
            print("exit early because empty timing_info")
            return
        self.results_arr.append({ k: pd.Timestamp(v) for k,v in self.timing_info.items()})
        if self.current_n > self.max_n:
            print("done")
            return
        else:
            print("setting arrow bytes")
            self.timing_info = {}
            time.sleep(.1)
            self.df_arrow_bytes=gen_func(self.current_n)
            print("after set arrow bytes")
            self.current_n += 1

results = []
sbw = CallbackBytesBenchmarkWidget(gen_func, results, min_n=1, max_n=7)
sbw

In [ ]:
simple_df = pd.DataFrame({'a':[99,27], 'b': ['paddy', 'margaret']})
N = 4000 
big_df = df = pd.DataFrame({
    'one': [1]*N,
    'str': ["foo", "barr", "bazz", "bofffff"] * (N//4),
    'log_normal': np.random.lognormal(25, .3, N)})

In [ ]:
class CallbackBytesBenchmarkWidget(BytesBenchmarkWidget):
    def __init__(self, cb, **kwargs):
        self.cb = cb
        super().__init__(**kwargs)

    
    @observe('timing_info')
    def _timing_info_change(self, change):
        self.cb()
clos_arr = []

timing_infos = []
def respond_to_cb():
    w = clos_arr[-1]
    timing_infos.append({ k: pd.Timestamp(v) for k,v in w.timing_info.items()})
    print("here")
    if len(timing_infos) < 5:
        print("calling display")
        #w.close()
        #del clos_arr[0]
        new_w = CallbackBytesBenchmarkWidget(respond_to_cb, df_arrow_bytes=df_to_arrow_bytes(simple_df))
        clos_arr.append(new_w)
        display(new_w)
clos_arr.append(CallbackBytesBenchmarkWidget(respond_to_cb, df_arrow_bytes=df_to_arrow_bytes(simple_df)))
display(clos_arr[0])

In [ ]:
class CallbackBytesBenchmarkWidget(BytesBenchmarkWidget):
    def __init__(self, gen_func, results_arr, min_n, max_n, **kwargs):
        self.current_n = min_n
        super().__init__(df_arrow_bytes=gen_func(self.current_n))
        self.current_n += 1
        
        
    @observe('timing_info')
    def _timing_info_change(self, change):
        self.results_arr.append({ k: pd.Timestamp(v) for k,v in w.timing_info.items()})
        self.df_arrow_bytes=gen_func(self.current_n))
        self.current_n += 1


clos_arr = []

timing_infos = []
def respond_to_cb():
    w = clos_arr[-1]
    timing_infos.append({ k: pd.Timestamp(v) for k,v in w.timing_info.items()})
    print("here")
    if len(timing_infos) < 5:
        print("calling display")
        #w.close()
        #del clos_arr[0]
        new_w = CallbackBytesBenchmarkWidget(respond_to_cb, df_arrow_bytes=df_to_arrow_bytes(simple_df))
        clos_arr.append(new_w)
        display(new_w)
clos_arr.append(CallbackBytesBenchmarkWidget(respond_to_cb, df_arrow_bytes=df_to_arrow_bytes(simple_df)))
display(clos_arr[0])

In [ ]:
10**1,10**2

In [ ]:
pd.DataFrame(results)

In [ ]:
len(gen_func(5))

In [ ]:
sbw = SimpleBytesBenchmarkWidget(df_arrow_bytes=df_to_arrow_bytes(simple_df))
display(sbw)

In [ ]:
sbw.timing_info

In [ ]:
del(sbw)

In [ ]:
sbw.close()

In [ ]:
sbw = SimpleBytesBenchmarkWidget(df_arrow_bytes=df_to_arrow_bytes(simple_df))
def worker():
    print('Worker started')
    time.sleep(3)
    print(sbw.timing_info)
    print('Worker finished')
display(sbw)

thread = threading.Thread(target=worker)
thread.start()

print('Main thread continues')
thread.join()
print(sbw.timing_info)

In [ ]:
widgets = []
for i in range(5):
    #time.sleep(.5)
    widgets.append(SimpleBytesBenchmarkWidget(df_arrow_bytes=df_to_arrow_bytes(simple_df)))
for x in widgets:
    display(x)
    while len(x.timing_info) == 0:
        time.sleep(.1)


In [ ]:
widgets[0].timing_info

In [ ]:
#display(widgets[0])
#time.sleep(1)
#widgets[0].timing_info

In [ ]:
tdf = pd.DataFrame([ { k: pd.Timestamp(v) for k,v in x.timing_info.items() } for x in widgets])
tdf['elapsed'] = tdf['t3'] - tdf['t1']
tdf

In [ ]:
#timing_infos = []
widgets = []
for i in range(5):
    widgets.append(SimpleBytesBenchmarkWidget(df_arrow_bytes=df_to_arrow_bytes(simple_df)))
    #display(sbw)
    #timing_infos.append(sbw)
    #while len(sbw.timing_info) == 0:
    #    print("sleep")
    #    time.sleep(.5)
    #print("found timing info", sbw.timing_info)
    #timing_infos.append(sbw.timing_info)
#pd.DataFrame(timing_infos)

In [ ]:
pd.DataFrame([x.timing_info for x in timing_infos])

In [ ]:
sbw.timing_info

In [ ]:
sbw = SimpleBytesBenchmarkWidget(df_arrow_bytes=df_to_arrow_bytes(simple_df))
display(sbw)
print(sbw.timing_info, len(sbw.timing_info))

In [ ]:
print(sbw.timing_info, len(sbw.timing_info))

In [ ]:
def format_td(td):
    return "{td.seconds}.{td.microseconds}".format(td=td)

class BytesBenchmarkWidget(BaseWidget):
    _view_name = Unicode('BytesBenchmarkWidgetView').tag(sync=True)
    df_arrow_bytes = Bytes().tag(sync=True)
    timing_info = Dict({}).tag(sync=True)

    def __init__(self, html_output, serial_t1, serial_t2, **kwargs):
        self.serial_t1, self.serial_t2 = serial_t1, serial_t2
        self.html_output = html_output
        super().__init__(**kwargs)
    
    @observe('timing_info')
    def _timing_info_change(self, change):
        self.js_t1 = pd.Timestamp(self.timing_info['t1'])
        self.js_t2 = pd.Timestamp(self.timing_info['t2'])
        self.js_t3 = pd.Timestamp(self.timing_info['t3'])
        self.t4 = pd.Timestamp.utcnow()
        
        self.timedeltas = dict(
            py_serialize = self.serial_t2 - self.serial_t1,
            widget_xfer = self.js_t1 - self.serial_t2,
            js_deserialize = self.js_t2 - self.js_t1,
            js_iterate = self.js_t3 - self.js_t2,
            py_notify = self.t4 - self.js_t3,
            roundtrip = self.t4 - self.serial_t1
        )
        template = """
        <table border="1">
            <thead>
                <th>python serialize</th>
                <th>widget transmit</th>
                <th>js deserialize</th>
                <th>js iterate</th>
                <th>python notify</th>
                <th>roundtrip</th>
            </thead>
            <tbody >
                <tr>
                    <td> {py_serialize} </td>
                    <td> {widget_xfer} </td>
                    <td> {js_deserialize} </td>
                    <td> {js_iterate} </td>
                    <td> {py_notify} </td>
                    <td> {roundtrip} </td>
                </tr>
            </tbody>
        </table>
        """
        self.html_output.value = template.format(
            **{k: format_td(v) for k,v in self.timedeltas.items()})

def benchmark(df):
    output_area = HTML(
        value="Hello <b>World</b>",
        placeholder='Some HTML2',
        description='Some HTML3')
    display(output_area)
    t1 = pd.Timestamp.utcnow()
    table = pa.Table.from_pandas(df)
    fake_file = io.BytesIO()
    feather.write_feather(table, fake_file, compression='uncompressed')
    fake_file.seek(0)
    full_bytes = fake_file.read()
    t2 = pd.Timestamp.utcnow()
    bbw = BytesBenchmarkWidget(output_area, df_arrow_bytes=full_bytes, serial_t1=t1, serial_t2=t2)
    return bbw


df = pd.DataFrame({'a':[99,27], 'b': ['paddy', 'margaret']})
bnch= benchmark(df)
bnch

In [ ]:
def df_to_arrow_bytes(df):
    table = pa.Table.from_pandas(df)
    fake_file = io.BytesIO()
    feather.write_feather(table, fake_file, compression='uncompressed')
    fake_file.seek(0)
    return fake_file.read()

class SimpleBytesBenchmarkWidget(BaseWidget):
    _view_name = Unicode('BytesBenchmarkWidgetView').tag(sync=True)
    df_arrow_bytes = Bytes().tag(sync=True)
    timing_info = Dict({}).tag(sync=True)

simple_df = pd.DataFrame({'a':[99,27], 'b': ['paddy', 'margaret']})
N = 4000 
big_df = df = pd.DataFrame({
    'one': [1]*N,
    'str': ["foo", "barr", "bazz", "bofffff"] * (N//4),
    'log_normal': np.random.lognormal(25, .3, N)})

#sbw = SimpleBytesBenchmarkWidget(df_arrow_bytes=df_to_arrow_bytes(simple_df))
def _timing_info_change(change):
    print(change['new'])
#sbw.observe(_timing_info_change, 'timing_info')

#print(sbw.timing_info)
#sbw.df_arrow_bytes=df_to_arrow_bytes(simple_df)
#display(sbw)

In [ ]:
sbw = SimpleBytesBenchmarkWidget(df_arrow_bytes=df_to_arrow_bytes(simple_df))
display(sbw)
print(sbw.timing_info, len(sbw.timing_info))

In [ ]:
print(sbw.timing_info, len(sbw.timing_info))

In [ ]:
sbw.close()

In [ ]:
timing_infos = []
for i in range(5):
    sbw = SimpleBytesBenchmarkWidget(df_arrow_bytes=df_to_arrow_bytes(simple_df))
    display(sbw)
    while len(sbw.timing_info) == 0:
        print("sleep")
        time.sleep(.5)
    print("found timing info", sbw.timing_info)
    timing_infos.append(sbw.timing_info)
pd.DataFrame(timing_infos)

In [ ]:
sbw.timing_info

In [ ]:
def df_to_arrow_bytes(df):
    table = pa.Table.from_pandas(df)
    fake_file = io.BytesIO()
    feather.write_feather(table, fake_file, compression='uncompressed')
    fake_file.seek(0)
    return fake_file.read()

In [ ]:
class SimpleBytesBenchmarkWidget(BaseWidget):
    _view_name = Unicode('BytesBenchmarkWidgetView').tag(sync=True)
    df_arrow_bytes = Bytes().tag(sync=True)
    timing_info = Dict({}).tag(sync=True)

def benchmark(df):
    output_area = HTML(
        value="Hello <b>World</b>",
        placeholder='Some HTML2',
        description='Some HTML3')
    display(output_area)
    t1 = pd.Timestamp.utcnow()

    t2 = pd.Timestamp.utcnow()
    bbw = BytesBenchmarkWidget(output_area, df_arrow_bytes=full_bytes, serial_t1=t1, serial_t2=t2)
    return bbw

    
    @observe('timing_info')
    def _timing_info_change(self, change):
        self.js_t1 = pd.Timestamp(self.timing_info['t1'])
        self.js_t2 = pd.Timestamp(self.timing_info['t2'])
        self.js_t3 = pd.Timestamp(self.timing_info['t3'])
        self.t4 = pd.Timestamp.utcnow()
        
        self.timedeltas = dict(
            py_serialize = self.serial_t2 - self.serial_t1,
            widget_xfer = self.js_t1 - self.serial_t2,
            js_deserialize = self.js_t2 - self.js_t1,
            js_iterate = self.js_t3 - self.js_t2,
            py_notify = self.t4 - self.js_t3,
            roundtrip = self.t4 - self.serial_t1
        )
        template = """
        <table border="1">
            <thead>
                <th>python serialize</th>
                <th>widget transmit</th>
                <th>js deserialize</th>
                <th>js iterate</th>
                <th>python notify</th>
                <th>roundtrip</th>
            </thead>
            <tbody >
                <tr>
                    <td> {py_serialize} </td>
                    <td> {widget_xfer} </td>
                    <td> {js_deserialize} </td>
                    <td> {js_iterate} </td>
                    <td> {py_notify} </td>
                    <td> {roundtrip} </td>
                </tr>
            </tbody>
        </table>
        """
        self.html_output.value = template.format(
            **{k: format_td(v) for k,v in self.timedeltas.items()})

def benchmark(df):
    output_area = HTML(
        value="Hello <b>World</b>",
        placeholder='Some HTML2',
        description='Some HTML3')
    display(output_area)
    t1 = pd.Timestamp.utcnow()
    table = pa.Table.from_pandas(df)
    fake_file = io.BytesIO()
    feather.write_feather(table, fake_file, compression='uncompressed')
    fake_file.seek(0)
    full_bytes = fake_file.read()
    t2 = pd.Timestamp.utcnow()
    bbw = BytesBenchmarkWidget(output_area, df_arrow_bytes=full_bytes, serial_t1=t1, serial_t2=t2)
    return bbw


df = pd.DataFrame({'a':[99,27], 'b': ['paddy', 'margaret']})
bnch= benchmark(df)
bnch

In [ ]:
pd.Timedelta("10 seconds")

In [ ]:
def stat_benchmark(df_f, min_n, max_n, steps=10, runs=7, max_time=pd.Timedelta("10 seconds")):
    

In [ ]:
N = 10_000_000
df = pd.DataFrame({
    'one': [1]*N,
    'str': ["foo", "barr", "bazz", "bofffff"] * (N//4),
    'log_normal2': np.random.lognormal(25, .3, N),
    'log_normal': np.random.lognormal(25, .3, N)})

In [ ]:
bbw = benchmark(df)
bbw

In [ ]:
ab = bbw.td2

In [ ]:
ab.seconds

In [ ]:
ab.microseconds

In [ ]:
bench

In [ ]:
ab = benchmark(df)
ab.timing_info

In [ ]:
ab.timing_info

In [ ]:
Base64Widget(df_base64=base64table)

In [ ]:
df = pd.DataFrame({'a':[99,27], 'b': ['paddy', 'margaret']})
#df = pd.DataFrame({'a':[10,530], 'c': [-3.5, 9.7], 'd': [1,2]})

table = pa.Table.from_pandas(df)
feather.write_feather(table, "simple_df.feath", compression='uncompressed')
feath_text = open("simple_df.feath", "rb").read()
usable_feath_text = base64.b64encode(feath_text).decode('utf8')

print(len(feath_text))
feath_text[:500]

In [ ]:
import io
table = pa.Table.from_pandas(df)
fake_file = io.BytesIO()
feather.write_feather(table, fake_file, compression='uncompressed')
#feath_text = open("simple_df.feath", "rb").read()
fake_file.seek(0)
BytesWidget(df_arrow_bytes=fake_file.read())

In [ ]:
BytesWidget(df_arrow_bytes=feath_text)

In [ ]:
usable_feath_text = base64.b64encode(feath_text).decode('utf8')
Base64Widget(df_base64=usable_feath_text)

In [ ]:
def B64Viewer(df):
    table = pa.Table.from_pandas(df)
    feather.write_feather(table, "simple_df.feath", compression='uncompressed')
    feath_text = open("simple_df.feath", "rb").read()
    usable_feath_text = base64.b64encode(feath_text).decode('utf8')
    usable_feath_text = base64.b64encode(feath_text).decode('utf8')
    return [Base64Widget(df_base64=usable_feath_text), usable_feath_text]
ab = B64Viewer(pd.DataFrame({'a': [1, None, 2, 3,-3]}))
ab[0]

In [ ]:
split_string(ab[1], 70)

In [ ]:
import numpy as np

In [ ]:
ab = B64Viewer(pd.DataFrame({'a': [1, None, np.nan, np.inf, np.inf * -1]}))
ab[0]

In [ ]:
split_string(ab[1], 70)

In [ ]:
split_string(usable_feath_text, 70)

In [ ]:
b64_t2 = base64.b64encode(feath_text)
print(b64_t2[:30])
print("  %s" % b64_t2.decode('utf8')[:30])

In [ ]:
b64_t2.decode('utf8')[::70]

In [ ]:
def split_string(text, stride):
    return [text[i:i+stride] for i in range(0, len(text), stride)]
#split_string("asdfasdfasdf", 3)
split_string(b64_t2.decode('utf8'), 70)

In [ ]:
df

In [ ]:
import base64
b64_t2 = base64.b64encode(feath_text)
b64_t2

In [ ]:
pa.fea

In [ ]:
sink = pa.BufferOutputStream()
buf = sink.getvalue()
with pa.ipc.open_stream(buf) as reader:
      schema = reader.schema
      batches = [b for b in reader]


In [ ]:
BytesWidget(df_arrow_bytes=pabuffer.to_pybytes())

In [ ]:
import base64
b64_t2 = base64.b64encode(pabuffer.to_pybytes())
#Base64Widget(df_base64=b64_t2)

In [ ]:
b64_t2

In [ ]:
b64_t3 = '/////8gAAAAUAAAAAAAAAAwAFgAGAAUACAAMAAwAAAAAAwQAGAAAADAAAAAAAAAAAAAKABgADAAEAAgACgAAAGwAAAAQAAAAAgAAAAAAAAAAAAAABQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAABAAAAAAAAAADAAAAAAAAAAgAAAAAAAAAA0AAAAAAAAAAAAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAgAAAAAAAAAAAAAAAAAAAAoAAAAAAAAAMgAAAAAAAAAAAAAABQAAAA0AAAAAAAAAcGFkZHltYXJnYXJldAAAAA=='

In [ ]:
Base64Widget(df_base64=b64_t3)

In [ ]:
len(b64_t2)

In [ ]:
pabuffer.size

In [ ]:
BytesWidget(df_arrow_bytes=pabuffer.to_pybytes())

In [ ]:
BytesWidget(df_arrow_bytes=pabuffer.to_pybytes())